In [2]:
import pandas as pd
import numpy as np
import pyodbc;
import random
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.cross_validation import StratifiedKFold
from scipy.sparse.linalg import svds

from IPython.core.display import display,HTML
display(HTML("<style>.container{width:90% !important;}</style>"))

conn = pyodbc.connect(r'DRIVER={SQL Server Native Client 11.0};'r'SERVER=WIN-5G5AUOCEJPK;'r'DATABASE=Gold585;'r'Trusted_Connection=yes;')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [62]:
sql_query = "SELECT Distinct MaterialID \
       ,[Name] contentId \
       ,[GroupID] \
       ,[CollectionID] \
       ,[WeightNet] \
       ,[WearType] \
       ,[Proba] \
       ,[PriceSegment] \
       ,[Metal] \
       ,[MetalColor] \
  FROM [Gold585].[dbo].[Purchase]"
articles_df = pd.read_sql(sql_query,conn)

In [3]:
sql_query = "SELECT [PurchaseDateTime] \
      ,[PartnerID] personId\
      ,[labels] \
      ,MaterialID contentId\
      ,GroupID \
      ,[Quantity] \
      ,[SummaAfterDiscount] eventStrength\
  FROM [Gold585].[dbo].[Purchase] \
  Where PartnerID <> 0 \
  order by [PurchaseDateTime]"
interactions_df = pd.read_sql(sql_query,conn)

In [4]:
users_interactions_count_df = interactions_df.groupby(['personId', 'contentId']).size().groupby('personId').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users: 2728974
# users with at least 5 interactions: 445819


In [5]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'personId',
               right_on = 'personId')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 9010742
# of interactions from users with at least 5 interactions: 4315695


In [6]:
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['personId', 'contentId'])['eventStrength'].sum().reset_index() \
#                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 3947678


,personId,contentId,eventStrength
0,4,13872,559.0
1,4,31486,359.0
2,4,32918,249.0
3,4,58583,379.0
4,4,105627,489.0
5,4,107756,1079.0
6,4,111566,269.0
7,4,132909,199.0
8,4,133695,1742.0
9,4,203294,1790.0


In [7]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['personId'], 
                                   test_size=0.25,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 2960758
# interactions on Test set: 986920


In [8]:
interactions_full_indexed_df = interactions_full_df.set_index('personId')
interactions_train_indexed_df = interactions_train_df.set_index('personId')
interactions_test_indexed_df = interactions_test_df.set_index('personId')

In [9]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['contentId']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [10]:
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(articles_df['contentId'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testsdet = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    interactions_train_indexed_df), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['contentId'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['contentId'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()    

In [11]:
item_popularity_df = interactions_full_df.groupby('contentId')['eventStrength'].sum().sort_values(ascending=False).reset_index()
item_popularity_df.head(10)

,contentId,eventStrength
0,139016,43480484.66
1,133458,33712278.32
2,124884,27042301.94
3,134607,22988662.99
4,124399,22621834.92
5,124886,18655367.47
6,134141,16162237.81
7,137405,16038791.70
8,71444,15768097.66
9,14642,15154367.22


In [12]:
#Первый вариант рекоменд системы

In [172]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df[~self.popularity_df['contentId'].isin(items_to_ignore)] \
                               .sort_values('eventStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['Name', 'GroupID', 'WearType', 'Metal', 'PriceSegment']]


        return recommendations_df
    
popularity_model = PopularityRecommender(item_popularity_df, articles_df)

In [ ]:
%%time
print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df = model_evaluator.evaluate_model(popularity_model)
print('\nGlobal metrics:\n%s' % pop_global_metrics)
pop_detailed_results_df.head(10)

Evaluating Popularity recommendation model...


# Matrix Factorization

In [16]:
interactions_train_df.head()

,personId,contentId,eventStrength
1524793,1682961,203960,423.20
732391,1126214,124900,7497.67
3640394,5200554,140512,99.00
3730013,5458895,9799,198.00
2312746,2565534,129196,199.00


In [39]:
sql_query ="SELECT Name contentId \
,PartnerID personId \
,SUM(Quantity) eventStrength \
FROM [Gold585].[dbo].[Purchase] \
Where PartnerID <> 0 and [PurchaseDateTime] < '20180601'\
and PartnerID in (Select DISTINCT PartnerID From [Gold585].[dbo].[Purchase] where PurchaseDateTime>'20160101') \
and GroupID IN (SELECT TOP 50 GroupID contentId FROM [Gold585].[dbo].[Purchase] Group By GroupID order by SUM([Quantity]) DESC) \
and GroupID <> 276 \
Group By Name, PartnerID"

data = pd.read_sql(sql_query,conn)

In [41]:
users_interactions_count_df = data.groupby(['personId', 'contentId']).size().groupby('personId').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 3].reset_index()[['personId']]
print('# users with at least 2 interactions: %d' % len(users_with_enough_interactions_df))

# users: 1957317
# users with at least 2 interactions: 591813


In [42]:
print('# of interactions: %d' % len(data))
interactions_from_selected_users_df = data.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'personId',
               right_on = 'personId')
print('# of interactions from users with at least 2 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 4944731
# of interactions from users with at least 2 interactions: 3118892


In [43]:
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['personId', 'contentId'])['eventStrength'].sum().reset_index() \
#                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 3118892


,personId,contentId,eventStrength
0,4,"Кольцо с жемч.и фиан., 16,5",1
1,4,"Кольцо с цирконами, 16,5",1
2,4,"Серьги, жемчуг и фиан., родир.",1
3,12,"БРАСЛЕТ РОМБ 18; ПУСТ; 0,50; 585 2ЦВ",1
4,12,Кольцо с фианитами 16,1
5,12,Крест с родир.,1
6,12,Серьги с фианитами,1
7,14,"Кольцо обр. гл. 19.0, БК (ш3)",1
8,14,Кольцо с бриллиантом 16,1
9,14,"ЦЕПЬ СИНГАПУР 45; ПОЛН; 0,20; 585 КР",1


In [44]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['personId'], 
                                   test_size=0.29,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 2214413
# interactions on Test set: 904479


In [45]:
interactions_train_df.shape

(2214413, 3)

In [49]:
from scipy.sparse import csr_matrix

user_u = list(sorted(interactions_train_df.personId.unique()))
item_u = list(sorted(interactions_train_df.contentId.unique()))

row = interactions_train_df.personId.astype('category', categories=user_u).cat.codes
col = interactions_train_df.contentId.astype('category', categories=item_u).cat.codes

data1 = interactions_train_df['eventStrength'].tolist()

sparse_matrix = csr_matrix((data1, (row, col)), shape=(len(user_u), len(item_u)))

df = pd.SparseDataFrame([ pd.SparseSeries(sparse_matrix[i].toarray().ravel(), fill_value=0) 
                              for i in np.arange(sparse_matrix.shape[0]) ], 
                       index=user_u, columns=item_u, default_fill_value=0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  import sys


In [50]:
df.head(10)

,(Блокирован) Икона Владимиская,(Блокирован) Икона Спаситель,Cерьги,Cерьги с топазом родиров.,"Cерьги SWAROVSKI, фиан., родир.",Cерьги винтовые с жемчугом,Cерьги с бриллиантами и жемчугом,"Cерьги с фиан дорож, 925 бел,220-5-018#","Cерьги, роз.кабошон, родир.","Авт брелок Chevr,925бел,000-5-234#925",...,цепь Фигаро 45.0,цепь бисмарк с чернением 55.0,цепь нонна пустотелая 40.0,цепь нонна пустотелая 45.0,цепь нонна пустотелая 50.0,цепь нонна пустотелая 55.0,цепь нонна пустотелая 60.0,цепь ромб двойной с алм.огран. 40.0,цепь ромб двойной с алм.огран. 50.0,цепь ромб двойной с алм.огран. 60.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
#users_items_pivot_matrix_df = interactions_train_df.pivot(index='personId', columns='contentId',values='eventStrength').fillna(0)
#users_items_pivot_matrix_df.head(10)

ValueError: negative dimensions are not allowed

In [51]:
users_items_pivot_matrix_df.shape

(696303, 278)

In [52]:
users_items_pivot_matrix = users_items_pivot_matrix_df.as_matrix()
users_items_pivot_matrix[:10]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [53]:
users_ids = list(users_items_pivot_matrix_df.index)
users_ids[:10]

[4, 12, 14, 20, 21, 27, 28, 34, 39, 42]

In [54]:
users_items_pivot_matrix_df.shape

(696303, 278)

In [55]:
#The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15
#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)

In [56]:
U.shape

(696303, 15)

In [57]:
Vt.shape

(15, 278)

In [58]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

In [59]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[ 3.26556861e-04, -5.21541925e-05, -1.24744696e-03, ...,
         2.33162921e-06,  6.53007422e-07,  1.52286800e-04],
       [-5.97324739e-04,  2.23930812e-03, -4.94660070e-02, ...,
        -3.02912420e-05, -1.34507823e-06,  3.77367269e-03],
       [-8.46599494e-05,  1.73706512e-04,  7.22651063e-02, ...,
         3.46249121e-05,  1.02594855e-06,  2.62996186e-03],
       ...,
       [ 8.73245682e-04,  8.46664762e-04,  1.06064821e-02, ...,
         4.03617657e-05, -1.39317786e-06, -3.51275640e-04],
       [ 1.47101778e-03, -6.71382847e-04, -1.40826979e-01, ...,
         1.25945481e-06,  4.47566322e-06,  2.28923385e-04],
       [ 3.17822906e-03, -1.68732298e-03, -8.93623021e-02, ...,
         2.15439273e-05,  7.05814621e-06,  1.58994463e-03]])

In [60]:
#Converting the reconstructed matrix back to a Pandas dataframe
cf_preds_df = pd.DataFrame(all_user_predicted_ratings, columns = users_items_pivot_matrix_df.columns, index=users_ids).transpose()
cf_preds_df.head(10)

,4,12,14,20,21,27,28,34,39,42,...,6523538,6524140,6533243,6540921,6544686,6566327,6569176,6570354,6579847,6582253
contentId,,,,,,,,,,,,,,,,,,,,,
0,0.000327,-0.000597,-0.000085,0.004108,-0.002381,-0.000858,0.002955,0.000622,-0.000085,0.001377,...,-0.002797,-0.001062,0.002153,0.000791,0.001609,0.001313,0.003415,0.000873,0.001471,0.003178
214,-0.000052,0.002239,0.000174,0.000737,0.001506,0.001479,0.004361,0.001853,0.000174,0.000352,...,0.003922,0.001151,-0.000779,0.000301,0.004057,0.000721,0.009401,0.000847,-0.000671,-0.001687
215,-0.001247,-0.049466,0.072265,-0.033513,0.012956,-0.015308,0.642850,0.094857,0.072265,0.089838,...,-0.020546,0.009614,-0.039702,0.061377,0.608912,-0.054936,0.969741,0.010606,-0.140827,-0.089362
216,0.015444,0.127021,-0.053521,-0.009166,-0.011160,0.015007,0.370005,0.075684,-0.053521,0.117932,...,0.024054,0.000708,0.075724,0.064818,0.370426,0.038283,0.612304,0.017962,0.016313,0.058274
217,0.001677,0.020332,0.040980,0.071998,0.020854,0.000995,0.072804,0.054290,0.040980,0.013778,...,0.007361,0.008493,0.005148,0.010586,0.068612,0.014573,0.126672,0.014567,-0.002930,0.006876
218,0.006783,0.083819,-0.010085,0.052188,-0.012943,0.011488,0.044864,0.023536,-0.010085,0.030590,...,0.020683,-0.003912,0.031104,0.017601,0.050589,0.048030,0.106464,0.014274,0.025478,0.045873
219,0.000075,0.001814,0.002868,0.001736,0.002398,0.000227,0.002879,0.003156,0.002868,0.000452,...,0.001269,0.000726,-0.000180,0.000347,0.002526,-0.000198,0.004805,0.000417,-0.000268,0.000191
221,0.000010,0.000412,-0.000063,0.000180,0.000096,0.000440,0.000174,0.000056,-0.000063,-0.000039,...,0.000823,0.000175,0.000101,0.000001,0.000125,0.000325,0.000667,0.000180,0.000097,-0.000182
222,0.010991,0.995366,0.003377,-0.000358,0.001265,0.995422,-0.026274,0.002037,0.003377,0.007978,...,1.993419,0.003104,-0.013851,0.032090,-0.028861,0.991568,1.995724,0.001842,-0.016443,-0.006258


In [61]:
cf_preds_df.T.to_csv('prreds10052018.csv')

In [191]:
len(cf_preds_df.columns)

1640474

In [192]:
cf_preds_df.shape

(292, 1640474)

In [71]:
pd.DataFrame(cf_preds_df[439].sort_values(ascending=False)) #.to_csv('5295513.csv')

,439
contentId,
276,2.002215
222,1.005481
1890,0.050688
279,0.049988
225,0.038304
242,0.037370
1891,0.026909
1876,0.024421
215,0.023967


In [195]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False) \
                                    .reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['contentId'].isin(items_to_ignore)] \
                               .sort_values('recStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['Name', 'GroupID', 'WearType', 'Metal', 'PriceSegment']]


        return recommendations_df
    
cf_recommender_model = CFRecommender(cf_preds_df, articles_df)


In [196]:
print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)
print('\nGlobal metrics:\n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)

Evaluating Collaborative Filtering (SVD Matrix Factorization) model...


KeyError: 2526867

In [3]:
#Начнем для определенного класса
sql_query ="SELECT Name contentId \
,PartnerID personId \
,SUM(Quantity) eventStrength \
FROM [Gold585].[dbo].[Purchase] \
Where labels = '2.3' \
and PartnerID in (Select DISTINCT PartnerID From [Gold585].[dbo].[Purchase] where PurchaseDateTime>'20160101') \
and GroupID IN (SELECT TOP 50 GroupID contentId FROM [Gold585].[dbo].[Purchase] Group By GroupID order by SUM([Quantity]) DESC) \
and GroupID <> 276 \
Group By Name, PartnerID"

data = pd.read_sql(sql_query,conn)

In [5]:
data.shape

(848575, 3)

In [4]:
data.head()

,contentId,personId,eventStrength
0,"Серьги-пуссеты Цветки, роз.фиан., родир.",3220907,1
1,"Кольцо с фианитом, 17",3400755,1
2,Футляр пенал узкий,3630384,1
3,"Серьги, с фиан.,родир.",994938,1
4,"Цепь Венеция, родир., 50",3513453,1


In [7]:
# Оставим только клиентов, минимум с 2мя транзакциями
users_interactions_count_df = data.groupby(['personId', 'contentId']).size().groupby('personId').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 2].reset_index()[['personId']]
print('# users with at least 2 interactions: %d' % len(users_with_enough_interactions_df))

# users: 232658
# users with at least 2 interactions: 212121


In [9]:
print('# of interactions: %d' % len(data))
interactions_from_selected_users_df = data.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'personId',
               right_on = 'personId')
print('# of interactions from users with at least 2 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 848575
# of interactions from users with at least 2 interactions: 828038


In [10]:
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['personId', 'contentId'])['eventStrength'].sum().reset_index() \
#                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 828038


,personId,contentId,eventStrength
0,4,"Бусина Буква O, черн.",1
1,4,"Бусина Водолей, черн., золч.",1
2,4,"Бусина Голубь мира, черн.",1
3,4,"Бусина Яблоко с эмал., родир.",1
4,4,"Бусина с голуб.стеклом, оксидир.",1
5,4,"Кольцо с жемч.и фиан., 16,5",1
6,4,"Кольцо с цирконами, 16,5",1
7,4,"Подвес, зел.ю.с., фиан., родир.",1
8,4,"Серьги, жемчуг и фиан., родир.",1
9,4,"Цепь Ромб тройной родир., 45",1


In [17]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['personId'], 
                                   test_size=0.26,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 612748
# interactions on Test set: 215290


In [18]:
%%time
from scipy.sparse import csr_matrix

user_u = list(sorted(interactions_train_df.personId.unique()))
item_u = list(sorted(interactions_train_df.contentId.unique()))

row = interactions_train_df.personId.astype('category', categories=user_u).cat.codes
col = interactions_train_df.contentId.astype('category', categories=item_u).cat.codes

data1 = interactions_train_df['eventStrength'].tolist()

sparse_matrix = csr_matrix((data1, (row, col)), shape=(len(user_u), len(item_u)))

df = pd.SparseDataFrame([pd.SparseSeries(sparse_matrix[i].toarray().ravel(), fill_value=0) 
                              for i in np.arange(sparse_matrix.shape[0]) ], 
                       index=user_u, columns=item_u, default_fill_value=0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  import sys


Wall time: 3h 21min 21s


In [62]:
df.head(10)

,(Блокирован) Икона Владимиская,(Блокирован) Икона Спаситель,Cерьги,"Cерьги SWAROVSKI, фиан., родир.",Cерьги винтовые с жемчугом,Cерьги с бриллиантами и жемчугом,"Cерьги с фиан дорож, 925 бел,220-5-018#","Cерьги, роз.кабошон, родир.","Авт брелок BMW, 925 бел, 002-5-242#925","Авт брелок Chevr,925бел,000-5-234#925",...,серьги-конго,серьги-конго 28.0,серьги-конго 30.0,серьги-конго 50.0,серьги-конго 70.0,цепь Кордовая 50.0,цепь Фигаро 40.0,цепь бисмарк с чернением 50.0,цепь нонна пустотелая 45.0,цепь нонна пустотелая 55.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
112,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
124,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
df.to_csv('IdU-idI-Rank -Labels 2.2 from20160101.csv')

KeyboardInterrupt: 

In [20]:
users_items_pivot_matrix_df = df

In [21]:
users_items_pivot_matrix_df.shape

(212121, 13585)

In [63]:
users_items_pivot_matrix = users_items_pivot_matrix_df.as_matrix()
users_items_pivot_matrix[:100]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [69]:
users_ids = list(users_items_pivot_matrix_df.index)
users_ids[:10]

[4, 15, 28, 34, 41, 55, 84, 112, 124, 135]

In [67]:
users_items_pivot_matrix.shape

(212121, 13585)

In [71]:
users_items_pivot_matrix = users_items_pivot_matrix.astype('f')

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [76]:
from scipy.sparse import coo_matrix
users_items_pivot_matrix = coo_matrix(users_items_pivot_matrix)

In [78]:
users_items_pivot_matrix.asfptype()

<212121x13585 sparse matrix of type '<class 'numpy.float64'>'
	with 612748 stored elements in COOrdinate format>

In [85]:
users_items_pivot_matrix = csc_matrix(users_items_pivot_matrix, dtype=float)

In [86]:
#The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15
#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)

In [87]:
print(U.shape)
print(Vt.shape)
sigma = np.diag(sigma)
print(sigma.shape)

(212121, 15)
(15, 13585)
(15, 15)


In [93]:
def chunking_dot(big_matrix, small_matrix, chunk_size=100):
    # Make a copy if the array is not already contiguous
    small_matrix = np.ascontiguousarray(small_matrix)
    R = np.empty((big_matrix.shape[0], small_matrix.shape[1]))
    for i in range(0, R.shape[0], chunk_size):
        end = i + chunk_size
        R[i:end] = np.dot(big_matrix[i:end], small_matrix)
    return R

In [95]:
def slower_dot (A, B):
    """Low-memory implementation of dot product"""
    #Assuming A and B are of the right type and size
    R = np.empty([A.shape[0], B.shape[1]])
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            R[i,j] = np.dot(A[i,:], B[:,j])
    return R

In [98]:
%%time
#all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings = slower_dot(slower_dot(U, sigma), Vt) 
all_user_predicted_ratings

MemoryError: 